In [1]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy.orm import sessionmaker
import pandas as pd

# 1.0 Get Data 

In [2]:
host = 'interview-2.ck1h5ksgzpiq.us-east-1.rds.amazonaws.com'  
porta = 3306
usuario = 'hotinterview'
senha = '6cT4jk9QWPhQC9KXWKDd'
schema = 'innodb'
tabela = 'sales_data'

url_conexao = f'mysql+pymysql://{usuario}:{senha}@{host}:{porta}/{schema}'

In [3]:
engine = create_engine(url_conexao, echo=True)  # Use "echo=True" para ver as mensagens de log

In [4]:
Session = sessionmaker(bind=engine)
session = Session()
session.close()

In [5]:
query = 'SELECT * FROM sales_data'
df = pd.read_sql(query, engine)

2023-05-31 10:43:51,272 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-05-31 10:43:51,273 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-31 10:43:51,680 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-05-31 10:43:51,681 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-31 10:43:51,886 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-05-31 10:43:51,889 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-31 10:43:52,225 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-31 10:43:52,226 INFO sqlalchemy.engine.Engine DESCRIBE `innodb`.`SELECT * FROM sales_data`
2023-05-31 10:43:52,230 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-31 10:43:52,347 INFO sqlalchemy.engine.Engine SELECT * FROM sales_data
2023-05-31 10:43:52,349 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-31 10:44:39,299 INFO sqlalchemy.engine.Engine ROLLBACK


# 2.0 Observações iniciais 

- purchase_id: Identificação da compra na Hotmart;
- product_id: Identificação do produto na Hotmart;
- affiliate_id: Identificação do afiliado na Hotmart;
- producer_id: Identificação do produtor na Hotmart;
- buyer_id: Identificação do comprador na Hotmart;
- purchase_date: Data e hora em que a compra foi realizada;
- product_creation_date: Data e hora em que o produto foi criado na Hotmart;
- product_category: categoria do produto na Hotmart. Exemplo: e-book,
software, curso online, e-tickets, etc.;
- product_niche: nicho de mercado que o produto faz parte. Exemplo:
educação, saúde e bem-estar, sexualidade, etc.;
- purchase_value: valor da compra. Esse dado, assim como nicho e categoria foi
codificado para manter a confidencialidade. O valor apresentado no dataset é
o z-score do valor real;
- affiliate_commission_percentual: percentual de comissão que o afiliado
receberá da compra;
- purchase_device: tipo de dispositivo utilizado no momento da compra, como:
Desktop, Mobile, Tablet, ou Outros;
- purchase_origin: endereço do site do qual a pessoa veio antes da compra. Por
exemplo, se uma pessoa veio do Facebook, Youtube, ou até mesmo de outra
página no site oficial do produto;
- is_origin_page_social_network: informa se essa compra veio de uma URL do
Facebook, Youtube, Instagram, Pinterest, ou Twitter.

## Algumas regras de negócio
- Quando a compra for feita diretamente pelo produtor, ou seja, quando não
houver afiliado intermediando a compra, o campo
affiliate_commission_percentual terá valor 0, e o campo affiliate_id será
igual ao producer_id;
- No campo purchase_origin nós apenas consideramos o host do site. Isso quer
dizer que, se uma pessoa veio do site www.meuproduto.com/promocoes,
esse campo só irá retornar o valor www.meuproduto.com;

Validação da primeira regra de negócio. Nesse caso não temos nenhum registro onde a compra foi feita
diretamente pelo produtor.

In [6]:
df[df['product_id'] == df['affiliate_id']]

,purchase_id,product_id,affiliate_id,producer_id,buyer_id,purchase_date,product_creation_date,product_category,product_niche,purchase_value,affiliate_commission_percentual,purchase_device,purchase_origin,is_origin_page_social_network,Venda


A coluna **purchase_origin** não possui um comportamento de host como **www.meuproduto.com**, mas irei considerar que a informação é essa mesmo. 

In [18]:
df['purchase_origin'].value_counts()

purchase_origin
Origin ef2b    330077
Origin 5187    167028
Origin adf0     77857
Origin 18eb     28693
Origin 3ade     13069
                ...  
Origin e51d         1
Origin d5eb         1
Origin 00ca         1
Origin 7475         1
Origin 290d         1
Name: count, Length: 9603, dtype: int64

In [ ]:
df

# DATA CLEANING 

In [21]:
df['affiliate_commission_percentual'].value_counts()

affiliate_commission_percentual
0.00     1338727
50.00      70931
60.00      33794
30.00      20419
40.00      19606
          ...   
29.60          1
29.48          1
39.03          1
30.11          1
28.88          1
Name: count, Length: 279, dtype: int64